<a href="https://colab.research.google.com/github/papichoolo/Soybeans/blob/main/SoyNet_Beans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mounting the dataset from Google Drive and importing libraries

In [43]:
import numpy as np
import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/content/drive/MyDrive/Camera Clicks_256_256')

print (dataset_path)  #what kinds of classes are in this dataset

print("Types of classes labels found: ", len(dataset_path))

['Disease_Preprocessing data', 'Healthy_Preprocessing data']
Types of classes labels found:  2


In [44]:
class_labels = []

for item in dataset_path:
 # Get all the file names
 all_classes = os.listdir('/content/drive/MyDrive/Camera Clicks_256_256' + '/' +item)
 print(all_classes)

 # Add them to the list
 for room in all_classes:
    class_labels.append((item, str('dataset_path' + '/' +item) + '/' + room))

['Soy (1060).JPG', 'Soy (106).JPG', 'Soy (1061).JPG', 'Soy (1062).JPG', 'Soy (1064).JPG', 'Soy (1063).JPG', 'Soy (1066).JPG', 'Soy (1065).JPG', 'Soy (1068).JPG', 'Soy (1067).JPG', 'Soy (1069).JPG', 'Soy (1070).JPG', 'Soy (1071).JPG', 'Soy (1072).JPG', 'Soy (1074).JPG', 'Soy (1073).JPG', 'Soy (1075).JPG', 'Soy (1076).JPG', 'Soy (1078).JPG', 'Soy (1077).JPG', 'Soy (1080).JPG', 'Soy (1079).JPG', 'Soy (1082).JPG', 'Soy (1081).JPG', 'Soy (1083).JPG', 'Soy (1084).JPG', 'Soy (1085).JPG', 'Soy (1086).JPG', 'Soy (1087).JPG', 'Soy (1088).JPG', 'Soy (1089).JPG', 'Soy (109).JPG', 'Soy (1090).JPG', 'Soy (1092).JPG', 'Soy (1091).JPG', 'Soy (1094).JPG', 'Soy (1093).JPG', 'Soy (1095).JPG', 'Soy (1096).JPG', 'Soy (1097).JPG', 'Soy (1098).JPG', 'Soy (1099).JPG', 'Soy (110).JPG', 'Soy (11).JPG', 'Soy (1101).JPG', 'Soy (1100).JPG', 'Soy (1102).JPG', 'Soy (1104).JPG', 'Soy (1105).JPG', 'Soy (1106).JPG', 'Soy (1107).JPG', 'Soy (1108).JPG', 'Soy (1109).JPG', 'Soy (111).JPG', 'Soy (1110).JPG', 'Soy (1111).JPG

Creating a dataframe of Images and labels

In [45]:
df = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])
print(df.head())
print(df.tail())

                       Labels  \
0  Disease_Preprocessing data   
1  Disease_Preprocessing data   
2  Disease_Preprocessing data   
3  Disease_Preprocessing data   
4  Disease_Preprocessing data   

                                               image  
0  dataset_path/Disease_Preprocessing data/Soy (1...  
1  dataset_path/Disease_Preprocessing data/Soy (1...  
2  dataset_path/Disease_Preprocessing data/Soy (1...  
3  dataset_path/Disease_Preprocessing data/Soy (1...  
4  dataset_path/Disease_Preprocessing data/Soy (1...  
                          Labels  \
1189  Healthy_Preprocessing data   
1190  Healthy_Preprocessing data   
1191  Healthy_Preprocessing data   
1192  Healthy_Preprocessing data   
1193  Healthy_Preprocessing data   

                                                  image  
1189  dataset_path/Healthy_Preprocessing data/Soy (9...  
1190  dataset_path/Healthy_Preprocessing data/Soy (7...  
1191  dataset_path/Healthy_Preprocessing data/Soy (9...  
1192  dataset_path/Hea

In [46]:
print("Total number of images in the dataset: ", len(df))

label_count = df['Labels'].value_counts()
print(label_count)

Total number of images in the dataset:  1194
Disease_Preprocessing data    1055
Healthy_Preprocessing data     139
Name: Labels, dtype: int64


Reading the Image and Resizing it to 224 x 224

In [47]:
import cv2
path = '/content/drive/MyDrive/Camera Clicks_256_256/'
dataset_path = os.listdir('/content/drive/MyDrive/Camera Clicks_256_256')

im_size = 224

images = []
labels = []

for i in dataset_path:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path) ]

    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

In [48]:
images = np.array(images)

images = images.astype('float32') / 255.0
images.shape

(1194, 224, 224, 3)

Encoding labels

In [49]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
y=df['Labels'].values
print(y)

y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
print (y)

['Disease_Preprocessing data' 'Disease_Preprocessing data'
 'Disease_Preprocessing data' ... 'Healthy_Preprocessing data'
 'Healthy_Preprocessing data' 'Healthy_Preprocessing data']
[0 0 0 ... 1 1 1]


In [50]:
y=y.reshape(-1,1)

from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y) #.toarray()
print(Y[:5])
print(Y[35:])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [51]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, y = shuffle(images, Y, random_state=1)


train_x, test_x, train_y, test_y = train_test_split(images, y, test_size=0.05, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(1134, 224, 224, 3)
(1134, 2)
(60, 224, 224, 3)
(60, 2)


Using EfficientNetB0

Using a Learning Rate scheduler to create a custom Learning rate

In [52]:
from keras.callbacks import LearningRateScheduler
def lr_schedule(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * 0.9


Trying to implement SGD Function

In [53]:
import keras
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)


WandB Metrics uploaded

In [54]:
!pip install wandb

In [55]:
import wandb
from wandb.keras import WandbMetricsLogger


In [56]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

NUM_CLASSES = 2
IMG_SIZE = 224
size = (IMG_SIZE, IMG_SIZE)


inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))


# Using model without transfer learning

outputs = EfficientNetB0(include_top=True, weights=None, classes=NUM_CLASSES)(inputs)

In [57]:
model = tf.keras.Model(inputs, outputs)

model.compile(optimizer='rmsprop', loss="binary_crossentropy", metrics=["accuracy","F1Score","Precision","Recall"], )

model.summary()

scheduler= LearningRateScheduler(lr_schedule)

hist = model.fit(train_x, train_y, validation_split=0.2, epochs=50, verbose=1, callbacks=[WandbMetricsLogger()])


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, 2)                 4052133   
 )                                                               
                                                                 
Total params: 4052133 (15.46 MB)
Trainable params: 4010110 (15.30 MB)
Non-trainable params: 42023 (164.16 KB)
_________________________________________________________________
Epoch 1/50
 6/29 [=====>........................] - ETA: 6s - loss: 4.5778 - accuracy: 0.5729 - f1_score: 0.4657 - precision: 0.5729 - recall: 0.5729

29/29 [==============================] - 45s 369ms/step - loss: 1.4937 - accuracy: 0.7541 - f1_score: 0.5411 - precision: 0.7541 - recall: 0.7541 - val_loss: 0.4060 - val_accuracy: 0.9075 - val_f1_score: 0.4758 - val_precision: 0.9075 - val_recall: 0.9075
Epoch 2/50
29/29 [==============================] - 8s 266ms/step - loss: 0.3833 - accuracy: 0.8600 - f1_score: 0.5812 - precision: 0.8600 - recall: 0.8600 - val_loss: 0.4299 - val_accuracy: 0.9075 - val_f1_score: 0.4758 - val_precision: 0.9075 - val_recall: 0.9075
Epoch 3/50
29/29 [==============================] - 8s 280ms/step - loss: 0.3381 - accuracy: 0.8677 - f1_score: 0.6106 - precision: 0.8677 - recall: 0.8677 - val_loss: 0.3193 - val_accuracy: 0.9075 - val_f1_score: 0.4758 - val_precision: 0.9075 - val_recall: 0.9075
Epoch 4/50
29/29 [==============================] - 8s 276ms/step - loss: 0.3059 - accuracy: 0.8688 - f1_score: 0.6237 - precision: 0.8688 - recall: 0.8688 - val_loss: 0.3179 - val_accuracy: 0.9075 - val_f1_score

accuracy of 95.59% in only 5 epochs?? yep thats learning rate scheduling :)))

In [1]:
preds = model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

NameError: name 'model' is not defined

In [59]:
predictions = (model.predict(test_x) > 0.5).astype("int32")
predictions

2/2 [==============================] - 1s 62ms/step


array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0]], dtype=int32)

In [60]:
bruh= model.predict(test_x)
print(bruh)

2/2 [==============================] - 0s 56ms/step
[[9.9956876e-01 4.3117438e-04]
 [9.9999988e-01 6.9412415e-08]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 9.0159935e-10]
 [1.0000000e+00 0.0000000e+00]
 [9.9193317e-01 8.0668200e-03]
 [9.9999857e-01 1.4785446e-06]
 [1.0000000e+00 8.0308221e-10]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 1.9592673e-35]
 [1.0000000e+00 5.4881578e-08]
 [9.9999988e-01 8.7206431e-08]
 [1.0000000e+00 8.0674031e-26]
 [1.0000000e+00 1.2486757e-16]
 [1.0000000e+00 5.0990430e-08]
 [9.9999976e-01 2.7642767e-07]
 [1.0000000e+00 7.0564898e-10]
 [1.0000000e+00 0.0000000e+00]
 [9.9999952e-01 5.0354629e-07]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 0.0000000e+00]
 [1.0000000e+00 3.5176627e-16]
 [9.9999928e-01 6.8915199e-07]
 [9.9998248e-01 1.7527414e-05]
 [1.0000000e+00 0.0000000e+00]
 [9.9999976e-01 2.9558862e-07]
 [1.0000000e+00 0.0000000e+00]
 [9.9998331e-01 1.6694798e-05]
 [1.0000000e+00 5.8324840e-30]
 [9.9999857e-01 1.4785446e-06]
 [1.8888237e-01 8.

TAKING UNSEEN SOYABEAN LEAF IMAGE


It predicts diseased leaf, which is correct.

In [61]:
f1= hist.history['f1_score']
f1_scores = []
prec_list=[]
rec_list=[]
for pair in f1:
    precision = pair[0]
    recall = pair[1]
    f1_ = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    f1_scores.append(f1_)
    prec_list.append(precision)
    rec_list.append(recall)

# Convert the list of F1 scores to a NumPy array
f1_scores_array = np.array(f1_scores)
prec_list= np.array(prec_list)
rec_list= np.array(rec_list)
print(f1_scores_array)
print(prec_list)
print(rec_list)

[0.36035626 0.38033282 0.44655416 0.47581787 0.6938855  0.79237175
 0.81936109 0.84592076 0.82503675 0.88027224 0.86579603 0.89158431
 0.88132325 0.90473169 0.8835335  0.90473169 0.90389218 0.88924221
 0.89966593 0.9154514  0.89785388 0.89715328 0.92097459 0.91980019
 0.89876931 0.90559516 0.9121993  0.92049242 0.91652657 0.90719792
 0.91725311 0.90389218 0.9255863  0.92950094 0.9154514  0.9139729
 0.91440865 0.9157848  0.9139729  0.9157848  0.89178724 0.92877182
 0.9243369  0.90985188 0.90639257 0.92303269 0.91980019 0.90131544
 0.92937787 0.91065051]
[0.8537705  0.9228901  0.9270073  0.9273948  0.9449598  0.9618226
 0.96189874 0.9661654  0.9638854  0.9730407  0.97132164 0.9748428
 0.9730069  0.97875    0.97509336 0.97875    0.9780565  0.97564024
 0.9774718  0.9805643  0.9775281  0.9768316  0.98254365 0.9818637
 0.9775     0.97943926 0.9799499  0.98184097 0.98125    0.97867006
 0.9812265  0.9780565  0.9831144  0.9844237  0.9805643  0.9806129
 0.9798742  0.9812734  0.9806129  0.9812734

In [62]:
import plotly.graph_objs as go
import numpy as np
epochs=list(range(1,51))

trace_accuracy = go.Scatter(x=epochs, y=hist.history['accuracy'], mode='lines', name='Accuracy')
trace_precision = go.Scatter(x=epochs, y=prec_list, mode='lines', name='Precision')
trace_recall = go.Scatter(x=epochs, y=rec_list, mode='lines', name='Recall')
trace_f1_score = go.Scatter(x=epochs, y=f1_scores_array, mode='lines', name='F1 Score')

layout = go.Layout(
    title='Metrics Across Epochs',
    xaxis=dict(title='Epochs'),
    yaxis=dict(title='Value'),
)

# Combining traces and layout
data = [trace_accuracy, trace_precision, trace_recall, trace_f1_score]
fig = go.Figure(data=data, layout=layout)

# Displaying the plot
fig.show()

## Inception V3


In [63]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np

from tensorflow.keras import layers


NUM_CLASSES = 2
IMG_SIZE = 299
size = (IMG_SIZE, IMG_SIZE)


inputs1 = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))


# Using model without transfer learning

outputs1 = InceptionV3(include_top=True, weights=None, classes=NUM_CLASSES)(inputs1)


In [64]:
im_size = 299

images1 = []
labels1 = []

for i in dataset_path:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path) ]

    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images1.append(img)
        labels1.append(i)

In [65]:
images1 = np.array(images1)

images1 = images1.astype('float32') / 255.0
images1.shape

(1194, 299, 299, 3)

In [66]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images1, Y = shuffle(images1, Y, random_state=1)


train_x, test_x, train_y, test_y = train_test_split(images1, Y, test_size=0.05, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(1134, 299, 299, 3)
(1134, 2)
(60, 299, 299, 3)
(60, 2)


In [ ]:
from keras import optimizers, losses, activations, models
model1 = tf.keras.Model(inputs1, outputs1)

model1.compile(optimizer=optimizers.SGD(lr=1e-4,
                                       momentum=0.9), loss="binary_crossentropy", metrics=["accuracy","F1Score"] )

model1.summary()

hist1 = model1.fit(train_x, train_y, epochs=50, verbose=1, callbacks=[scheduler])

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 2)                 21806882  
                                                                 
Total params: 21806882 (83.19 MB)
Trainable params: 21772450 (83.06 MB)
Non-trainable params: 34432 (134.50 KB)
_________________________________________________________________


In [ ]:
preds = model1.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
hist1.history


Making F1 Score from scratch since the hist array is giving a combination of precision and recall

In [ ]:
f1= hist1.history['f1_score']
f1_scores = []
prec_list=[]
rec_list=[]
for pair in f1:
    precision = pair[0]
    recall = pair[1]
    f1_ = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    f1_scores.append(f1_)
    prec_list.append(precision)
    rec_list.append(recall)

# Convert the list of F1 scores to a NumPy array
f1_scores_array = np.array(f1_scores)
prec_list= np.array(prec_list)
rec_list= np.array(rec_list)
print(f1_scores_array)
print(prec_list)
print(rec_list)

Metrics

In [ ]:
import plotly.graph_objs as go
import numpy as np
epochs=list(range(1,51))

trace_accuracy = go.Scatter(x=epochs, y=hist1.history['accuracy'], mode='lines', name='Accuracy')
trace_precision = go.Scatter(x=epochs, y=prec_list, mode='lines', name='Precision')
trace_recall = go.Scatter(x=epochs, y=rec_list, mode='lines', name='Recall')
trace_f1_score = go.Scatter(x=epochs, y=f1_scores_array, mode='lines', name='F1 Score')

layout = go.Layout(
    title='Metrics Across Epochs',
    xaxis=dict(title='Epochs'),
    yaxis=dict(title='Value'),
)

# Combining traces and layout
data = [trace_accuracy, trace_precision, trace_recall, trace_f1_score]
fig = go.Figure(data=data, layout=layout)

# Displaying the plot
fig.show()